In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score, r2_score
import sys
sys.path.append(r"C:\Users\Talha Ali\Desktop\machine learning labs\timeseires")

In [3]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score, r2_score
from timeseires.utils.to_split import to_split
from timeseires.utils.multivariate_multi_step import multivariate_multi_step
from timeseires.utils.multivariate_single_step import multivariate_single_step
from timeseires.utils.univariate_multi_step import univariate_multi_step
from timeseires.utils.univariate_single_step import univariate_single_step
from timeseires.utils.CosineAnnealingLRS import CosineAnnealingLRS
from timeseires.callbacks.EpochCheckpoint import EpochCheckpoint
from tensorflow.keras.callbacks import ModelCheckpoint
from timeseires.callbacks.TrainingMonitor import TrainingMonitor
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import LSTM, Bidirectional, Add
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv1D,TimeDistributed
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten,MaxPooling1D,Concatenate,AveragePooling1D, GlobalMaxPooling1D, Input
from tensorflow.keras.models import Sequential,Model
import pandas as pd
import time, pickle
import numpy as np
import tensorflow.keras.backend as K
import tensorflow
from tensorflow.keras.layers import Input, Reshape, Lambda
from tensorflow.keras.layers import Layer, Flatten, LeakyReLU, concatenate, Dense
from tensorflow.keras.regularizers import l2
import glob
import h5py
import matplotlib.pyplot as plt
from keras.callbacks import Callback

In [5]:
#lookback = 24
model = None
start_epoch = 0
time_steps=24
num_features=21

In [7]:
def CNN():
    input_data = Input(shape=(time_steps, num_features))
    x1 = Conv1D(16, 2, activation="relu")(input_data)
    x2 = Conv1D(16, 2, activation="relu")(x1)
    flatten = Flatten()(x2)
    output_data = Dense(1)(flatten)
    model = Model(input_data, output_data)
    return model

In [9]:
model1 = CNN()
model1.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 24, 21)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d (Conv1D)                      │ (None, 23, 16)              │             688 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_1 (Conv1D)                    │ (None, 22, 16)              │             528 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 352)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │             353 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,569 (6.13 KB)

 Trainable params: 1,569 (6.13 KB)

 Non-trainable params: 0 (0.00 B)

In [11]:
tensorflow.keras.utils.plot_model(model1 )

You must install pydot (`pip install pydot`) for `plot_model` to work.


In [ ]:
checkpoints = r'C:\Users\Talha Ali\Desktop\machine learning labs\\E1-cp-{epoch:04d}-loss{val_loss:.2f}.h5'
OUTPUT_PATH = r'C:\Users\Talha Ali\Desktop\machine learning labs'
#FIG_PATH = os.path.sep.join([OUTPUT_PATH,"\history.png"])
#JSON_PATH = os.path.sep.join([OUTPUT_PATH,"\history.json"])

In [39]:
import os
from keras.callbacks import ModelCheckpoint

# Define necessary paths and variables
output_dir = "output"
os.makedirs(output_dir, exist_ok=True)

FIG_PATH = os.path.sep.join([output_dir, "training_plot.png"])
JSON_PATH = os.path.sep.join([output_dir, "training_history.json"])
checkpoints = os.path.join(r'C:\Users\PMLS\ml\lab8', 'best_model.h5')

start_epoch = 0  # Change this if you're resuming from a checkpoint

# Construct the callback to save only the *best* model to disk based on the validation loss
EpochCheckpoint1 = ModelCheckpoint(checkpoints,
                                   monitor="val_loss",
                                   save_best_only=True, 
                                   verbose=1)

# Make sure the TrainingMonitor class is implemented/imported properly
TrainingMonitor1 = TrainingMonitor(FIG_PATH, jsonPath=JSON_PATH, startAt=start_epoch)

# Construct the set of callbacks
callbacks = [EpochCheckpoint1, TrainingMonitor1]


In [ ]:
from keras.models import load_model
from keras.optimizers import Adam
import keras.backend as K

# Define your model path
model_path =r'C:\Users\Talha Ali\Desktop\machine learning labs\best_model.h5'

# if there is no specific model checkpoint supplied, then initialize
# the network and compile the model
if not os.path.exists(model_path):  # If the model file doesn't exist, start fresh
    print("[INFO] compiling model...")
    model = CNN()  # define your CNN() model earlier
    opt = Adam(1e-3)
    model.compile(loss='mae', optimizer=opt, metrics=["mae", "mape"])

# otherwise, load the checkpoint from disk
else:
    print(f"[INFO] loading model from {model_path}...")
    model = load_model(model_path)

    # update the learning rate
    print("[INFO] old learning rate: {}".format(K.get_value(model.optimizer.lr)))
    K.set_value(model.optimizer.lr, 1e-4)
    print("[INFO] new learning rate: {}".format(K.get_value(model.optimizer.lr)))


[INFO] compiling model...


In [ ]:
import os
path_dataset =r'C:\Users\Talha Ali\Desktop\machine learning labs'
path_tr = os.path.join(path_dataset, 'AEP_train.csv')
df_tr = pd.read_csv(path_tr)
train_set = df_tr.iloc[:].values
path_v = os.path.join(path_dataset, 'AEP_validation.csv')
df_v = pd.read_csv(path_v)
validation_set = df_v.iloc[:].values 
path_te = os.path.join(path_dataset, 'AEP_test.csv')
df_te = pd.read_csv(path_te)
test_set = df_te.iloc[:].values 

path_scaler = os.path.join(path_dataset, 'AEP_Scaler.pkl')
scaler         = pickle.load(open(path_scaler, 'rb'))

train_set.shape, validation_set.shape, test_set.shape

C:\Users\PMLS\anaconda3\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator MinMaxScaler from version 1.0.2 when using version 1.4.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


((84907, 21), (24259, 21), (12130, 21))

In [49]:
time_steps=24
num_features=21

In [51]:
start = time.time()
train_X , train_y = univariate_multi_step(train_set, time_steps, target_col=0,target_len=1)
validation_X, validation_y = univariate_multi_step(validation_set, time_steps, target_col=0,target_len=1)
test_X, test_y = univariate_multi_step(test_set, time_steps, target_col=0,target_len=1)
print('Time Consumed', time.time()-start, "sec")

Time Consumed 0.315476655960083 sec


In [53]:
epochs = 30
verbose = 1 #0
batch_size = 32
History = model.fit(train_X,
                        train_y,
                        batch_size=batch_size,   
                        epochs = epochs, 
                        validation_data = (validation_X,validation_y),
                        callbacks=callbacks,verbose = verbose)

Epoch 1/30
2615/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0682 - mae: 0.0682 - mape: 116.5950
Epoch 1: val_loss improved from inf to 0.02648, saving model to C:\Users\PMLS\ml\lab8\best_model.h5


2653/2653 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - loss: 0.0679 - mae: 0.0679 - mape: 132.3840 - val_loss: 0.0265 - val_mae: 0.0265 - val_mape: 12.9212
Epoch 2/30
2649/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0247 - mae: 0.0247 - mape: 1606.7852
Epoch 2: val_loss improved from 0.02648 to 0.02208, saving model to C:\Users\PMLS\ml\lab8\best_model.h5


2653/2653 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - loss: 0.0246 - mae: 0.0246 - mape: 1605.0803 - val_loss: 0.0221 - val_mae: 0.0221 - val_mape: 10.2605
Epoch 3/30
2624/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0183 - mae: 0.0183 - mape: 460.5322
Epoch 3: val_loss improved from 0.02208 to 0.01517, saving model to C:\Users\PMLS\ml\lab8\best_model.h5


2653/2653 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 0.0182 - mae: 0.0182 - mape: 458.7114 - val_loss: 0.0152 - val_mae: 0.0152 - val_mape: 7.1399
Epoch 4/30
2645/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0149 - mae: 0.0149 - mape: 59.1815
Epoch 4: val_loss improved from 0.01517 to 0.01377, saving model to C:\Users\PMLS\ml\lab8\best_model.h5


2653/2653 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 0.0149 - mae: 0.0149 - mape: 60.5700 - val_loss: 0.0138 - val_mae: 0.0138 - val_mape: 6.1374
Epoch 5/30
2639/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0130 - mae: 0.0130 - mape: 551.6251
Epoch 5: val_loss improved from 0.01377 to 0.01072, saving model to C:\Users\PMLS\ml\lab8\best_model.h5


...

2653/2653 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - loss: 0.0087 - mae: 0.0087 - mape: 93.0289 - val_loss: 0.0079 - val_mae: 0.0079 - val_mape: 3.9013
Epoch 23/30
2625/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0086 - mae: 0.0086 - mape: 86.8688
Epoch 23: val_loss did not improve from 0.00788
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - loss: 0.0086 - mae: 0.0086 - mape: 88.0051 - val_loss: 0.0091 - val_mae: 0.0091 - val_mape: 4.6139
Epoch 24/30
2638/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0086 - mae: 0.0086 - mape: 55.0711
Epoch 24: val_loss did not improve from 0.00788
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - loss: 0.0086 - mae: 0.0086 - mape: 56.3901 - val_loss: 0.0100 - val_mae: 0.0100 - val_mape: 5.5110
Epoch 25/30
2636/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0084 - mae: 0.0084 - mape: 328.0717
Epoch 25: val_loss did not improve from 0.00788
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - loss: 0.0084 - mae: 0.0084 - mape: 328.0383 - val_loss: 0.0082 - val_mae: 0.00

2653/2653 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - loss: 0.0083 - mae: 0.0083 - mape: 113.7773 - val_loss: 0.0079 - val_mae: 0.0079 - val_mape: 4.1231
Epoch 27/30
2635/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0084 - mae: 0.0084 - mape: 33.5740
Epoch 27: val_loss improved from 0.00786 to 0.00772, saving model to C:\Users\PMLS\ml\lab8\best_model.h5


2653/2653 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - loss: 0.0084 - mae: 0.0084 - mape: 34.9957 - val_loss: 0.0077 - val_mae: 0.0077 - val_mape: 3.9484
Epoch 28/30
2650/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0084 - mae: 0.0084 - mape: 79.6215
Epoch 28: val_loss improved from 0.00772 to 0.00761, saving model to C:\Users\PMLS\ml\lab8\best_model.h5


2653/2653 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - loss: 0.0084 - mae: 0.0084 - mape: 79.8714 - val_loss: 0.0076 - val_mae: 0.0076 - val_mape: 4.2383
Epoch 29/30
2624/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0085 - mae: 0.0085 - mape: 381.1940
Epoch 29: val_loss did not improve from 0.00761
2653/2653 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - loss: 0.0085 - mae: 0.0085 - mape: 380.1669 - val_loss: 0.0082 - val_mae: 0.0082 - val_mape: 4.0180
Epoch 30/30
2648/2653 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0082 - mae: 0.0082 - mape: 301.4902
Epoch 30: val_loss improved from 0.00761 to 0.00760, saving model to C:\Users\PMLS\ml\lab8\best_model.h5


2653/2653 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - loss: 0.0082 - mae: 0.0082 - mape: 301.0797 - val_loss: 0.0076 - val_mae: 0.0076 - val_mape: 3.6999


In [ ]:

model = load_model(r'C:\Users\Talha Ali\Desktop\machine learning labs\best_model.h5')

y_pred_scaled   = model.predict(test_X)
y_pred          = scaler.inverse_transform(y_pred_scaled)
y_test_unscaled = scaler.inverse_transform(test_y)
# Mean Absolute Error (MAE)
MAE = np.mean(abs(y_pred - y_test_unscaled)) 
print('Mean Absolute Error (MAE): ' + str(np.round(MAE, 2)))

# Median Absolute Error (MedAE)
MEDAE = np.median(abs(y_pred - y_test_unscaled))
print('Median Absolute Error (MedAE): ' + str(np.round(MEDAE, 2)))

# Mean Squared Error (MSE)
MSE = np.square(np.subtract(y_pred, y_test_unscaled)).mean()
print('Mean Squared Error (MSE): ' + str(np.round(MSE, 2)))

# Root Mean Squarred Error (RMSE) 
RMSE = np.sqrt(np.mean(np.square(y_pred - y_test_unscaled)))
print('Root Mean Squared Error (RMSE): ' + str(np.round(RMSE, 2)))

# Mean Absolute Percentage Error (MAPE)
MAPE = np.mean((np.abs(np.subtract(y_test_unscaled, y_pred)/ y_test_unscaled))) * 100
print('Mean Absolute Percentage Error (MAPE): ' + str(np.round(MAPE, 2)) + ' %')

# Median Absolute Percentage Error (MDAPE)
MDAPE = np.median((np.abs(np.subtract(y_test_unscaled, y_pred)/ y_test_unscaled))) * 100
print('Median Absolute Percentage Error (MDAPE): ' + str(np.round(MDAPE, 2)) + ' %')

print('\n\ny_test_unscaled.shape= ',y_test_unscaled.shape)
print('y_pred.shape= ',y_pred.shape)

379/379 [==============================] - 1s 1ms/step
Mean Absolute Error (MAE): 115.8
Median Absolute Error (MedAE): 93.33
Mean Squared Error (MSE): 23421.23
Root Mean Squared Error (RMSE): 153.04
Mean Absolute Percentage Error (MAPE): 0.8 %
Median Absolute Percentage Error (MDAPE): 0.64 %


y_test_unscaled.shape=  (12105, 1)
y_pred.shape=  (12105, 1)


In [ ]:
checkpoints = r'C:\Users\Talha Ali\Desktop\machine learning labs\E2-cp-{epoch:04d}-loss{val_loss:.2f}.h5'
model=r'C:\Users\Talha Ali\Desktop\machine learning labs\E1-cp-0058-loss0.01.h5'
start_epoch= 58

In [17]:
# construct the callback to save only the *best* model to disk
# based on the validation loss
EpochCheckpoint1 = ModelCheckpoint(checkpoints,
                             monitor="val_loss",
                             save_best_only=True, 
                             verbose=1)
TrainingMonitor1=TrainingMonitor(FIG_PATH, jsonPath=JSON_PATH, startAt=start_epoch)

# construct the set of callbacks
callbacks = [EpochCheckpoint1,TrainingMonitor1]
# if there is no specific model checkpoint supplied, then initialize
# the network and compile the model
if model is None:
    print("[INFO] compiling model...")
    model = PC.build(time_steps=24, num_features=21, reg=0.0005)
    opt = Adam(1e-3)
    model.compile(loss= 'mae', optimizer=opt, metrics=["mae", "mape"])
# otherwise, load the checkpoint from disk
else:
    print("[INFO] loading {}...".format(model))
    model = load_model(model)

    # update the learning rate
    print("[INFO] old learning rate: {}".format(K.get_value(model.optimizer.lr)))
    K.set_value(model.optimizer.lr, 1e-4)
    print("[INFO] new learning rate: {}".format(K.get_value(model.optimizer.lr)))

[INFO] loading C:\Users\Administrator\Downloads\ML Lab\checkpoint\ML Lab\lab8\E1-cp-0058-loss0.01.h5...
[INFO] old learning rate: 0.0010000000474974513
[INFO] new learning rate: 9.999999747378752e-05


In [18]:
epochs = 10
verbose = 1 #0
batch_size = 32
History = model.fit(train_X,
                        train_y,
                        batch_size=batch_size,   
                        epochs = epochs, 
                        validation_data = (validation_X,validation_y),
                        callbacks=callbacks,
                        verbose = verbose)

Epoch 1/10
2651/2653 [============================>.] - ETA: 0s - loss: 0.0064 - mae: 0.0064 - mape: 711.7792
Epoch 1: val_loss improved from inf to 0.00659, saving model to C:\Users\Administrator\Downloads\ML Lab\checkpoint\ML Lab\lab8\E2-cp-0001-loss0.01.h5
2653/2653 [==============================] - 36s 13ms/step - loss: 0.0064 - mae: 0.0064 - mape: 711.3610 - val_loss: 0.0066 - val_mae: 0.0066 - val_mape: 4.3460
Epoch 2/10
2644/2653 [============================>.] - ETA: 0s - loss: 0.0063 - mae: 0.0063 - mape: 578.1265
Epoch 2: val_loss improved from 0.00659 to 0.00642, saving model to C:\Users\Administrator\Downloads\ML Lab\checkpoint\ML Lab\lab8\E2-cp-0002-loss0.01.h5
2653/2653 [==============================] - 10s 4ms/step - loss: 0.0063 - mae: 0.0063 - mape: 576.2668 - val_loss: 0.0064 - val_mae: 0.0064 - val_mape: 4.2309
Epoch 3/10
2646/2653 [============================>.] - ETA: 0s - loss: 0.0063 - mae: 0.0063 - mape: 551.6437
Epoch 3: val_loss did not improve from 0.0064

In [ ]:

model = load_model(r'C:\Users\Talha Ali\Desktop\machine learning labs\checkpoint\\E2-cp-0008-loss0.01.h5')

y_pred_scaled   = model.predict(test_X)
y_pred          = scaler.inverse_transform(y_pred_scaled)
y_test_unscaled = scaler.inverse_transform(test_y)
# Mean Absolute Error (MAE)
MAE = np.mean(abs(y_pred - y_test_unscaled)) 
print('Mean Absolute Error (MAE): ' + str(np.round(MAE, 2)))

# Median Absolute Error (MedAE)
MEDAE = np.median(abs(y_pred - y_test_unscaled))
print('Median Absolute Error (MedAE): ' + str(np.round(MEDAE, 2)))

# Mean Squared Error (MSE)
MSE = np.square(np.subtract(y_pred, y_test_unscaled)).mean()
print('Mean Squared Error (MSE): ' + str(np.round(MSE, 2)))

# Root Mean Squarred Error (RMSE) 
RMSE = np.sqrt(np.mean(np.square(y_pred - y_test_unscaled)))
print('Root Mean Squared Error (RMSE): ' + str(np.round(RMSE, 2)))

# Mean Absolute Percentage Error (MAPE)
MAPE = np.mean((np.abs(np.subtract(y_test_unscaled, y_pred)/ y_test_unscaled))) * 100
print('Mean Absolute Percentage Error (MAPE): ' + str(np.round(MAPE, 2)) + ' %')

# Median Absolute Percentage Error (MDAPE)
MDAPE = np.median((np.abs(np.subtract(y_test_unscaled, y_pred)/ y_test_unscaled))) * 100
print('Median Absolute Percentage Error (MDAPE): ' + str(np.round(MDAPE, 2)) + ' %')

print('\n\ny_test_unscaled.shape= ',y_test_unscaled.shape)
print('y_pred.shape= ',y_pred.shape)

379/379 [==============================] - 1s 2ms/step
Mean Absolute Error (MAE): 103.3
Median Absolute Error (MedAE): 82.18
Mean Squared Error (MSE): 18772.44
Root Mean Squared Error (RMSE): 137.01
Mean Absolute Percentage Error (MAPE): 0.71 %
Median Absolute Percentage Error (MDAPE): 0.57 %


y_test_unscaled.shape=  (12105, 1)
y_pred.shape=  (12105, 1)
